In [ ]:
import os
import numpy as np
import pickle
import shutil 
import pImpactR as impact
import matplotlib.pyplot as plt
import time

# Simulation Setting

In [ ]:
npt = 1024
nTurn = 2048
Espread = 2.0e-3
pipe_radius = 0.029
std=0.2
Rsphere = 0.5

# Prepare Lattice

In [ ]:
beam,lattice=impact.readInputFile('IOTA.Chris.in')
beam.nCore_y = 4
beam.nCore_z = 4

In [ ]:
lattice = [ item for item in lattice if not item.type == 'write_raw_ptcl' ]
lattice = [ item for item in lattice if not item.type == '-8' ]
lattice[0].turns = 1
for i in range(len(lattice)):
    if 'length' in lattice[i].keys():
        lattice[i].n_sckick = int(np.ceil(lattice[i].length*40))
        lattice[i].n_map = 1
    if 'pipe_radius' in lattice[i].keys() and lattice[i].type!='dipole':
        lattice[i].pipe_radius = pipe_radius

In [ ]:
loop = lattice[0]
loop.turns = nTurn
nlinsert = lattice[1]
nlinsert.n_sckick = 2
nlinsert.n_map = 50
beam.n_particles = npt

In [ ]:
NL_nu = nlinsert.tune_advance
NL_L  = nlinsert.length
NL_c  = nlinsert.transverse_scale_c
NL_t  = nlinsert.strength_t
alfx = np.tan(np.pi*NL_nu)
betx = NL_L/np.sin(2.0*np.pi*NL_nu)
k = 2*alfx/betx
k

In [ ]:
nlinsert

In [ ]:
ke = beam.kinetic_energy
freq = beam.frequency
mass = beam.mass
gam0 = 1.0+beam.kinetic_energy/mass
bet0 = np.sqrt(1.0-1.0/gam0**2)
bg0  = np.sqrt(gam0**2-1.0)
q_m  = beam.multi_charge.q_m[0]

In [ ]:
elemTBT = impact.getElem('TBT_integral')
elemTBT.strength_t = NL_t
elemTBT.transverse_scale_c = NL_c
elemTBT.betx = betx
elemTBT.alfx = alfx
elemTBT.file_id = 10
elemTBT.pID_begin = 1
elemTBT.pID_end = npt

elemTBT_old = impact.getElem('TBT_integral_onMomentum')
elemTBT_old.strength_t = NL_t
elemTBT_old.transverse_scale_c = NL_c
elemTBT_old.betx = betx
elemTBT_old.alfx = alfx
elemTBT_old.file_id = 11
elemTBT_old.pID_begin = 1
elemTBT_old.pID_end = npt

lattice.append(elemTBT)
lattice.append(elemTBT_old)

# prepare particles

In [ ]:
def Impact2norm(data_in,bg0,bet0,sign=1):
    data=data_in.copy()
    data[:,5] = -(np.sqrt(1.0-2.0*data[:,5]/mass/(bet0*bg0)+(data[:,5]/mass)**2/bg0**2)-1.0)
    data[:,1] = (data[:,0]*alfx*sign/np.sqrt(betx) + data[:,1]/(1+data[:,5])*np.sqrt(betx))/NL_c
    data[:,3] = (data[:,2]*alfx*sign/np.sqrt(betx) + data[:,3]/(1+data[:,5])*np.sqrt(betx))/NL_c
    data[:,0] = data[:,0]/(np.sqrt(betx)*NL_c)
    data[:,2] = data[:,2]/(np.sqrt(betx)*NL_c)
    return data
    
def norm2Impact(data_in,bg0,bet0,sign=1):
    data=data_in.copy()
    data[:,1] = (-data[:,0]*alfx*sign + data[:,1])*NL_c/np.sqrt(betx)*(1+data[:,5])
    data[:,3] = (-data[:,2]*alfx*sign + data[:,3])*NL_c/np.sqrt(betx)*(1+data[:,5])
    data[:,0] = data[:,0]*np.sqrt(betx)*NL_c
    data[:,2] = data[:,2]*np.sqrt(betx)*NL_c
    data[:,5] = (bg0*np.sqrt(1/bet0**2+2.0*data[:,5]+data[:,5]**2)-bg0/bet0)*mass
    return data

In [ ]:
pData=np.zeros([npt,9])
pData[:,6]  = q_m
pData[:,-1] = np.arange(1,npt+1)

x = np.random.random([npt,4])
c0 = np.cos(x[:,0])
s0 = np.sin(x[:,0])
c1 = np.cos(x[:,1])
s1 = np.sin(x[:,1])
c2 = np.cos(x[:,2])
s2 = np.sin(x[:,2])
pData[:,0] = Rsphere*c0
pData[:,1] = Rsphere*s0*c1
pData[:,2] = Rsphere*s0*s1*c2
pData[:,3] = Rsphere*s0*s1*s2
pData[:,5] = (x[:,3]-0.5)*Espread

impact.writeParticleData(norm2Impact(pData,bg0,bet0,1),ke,mass,freq)

# Run

In [ ]:
def getLost():
    with open('lost_partcl.data') as f:
        for i, l in enumerate(f):
            pass
    return i

In [ ]:
impact.writeInputFile(beam,lattice)
impact.run(beam)
nLost0 = getLost()
indOff0,TBToff0 = impact.readTBT_integral(10)
indOn0,TBTon0 = impact.readTBT_integral_onMomentum(11)

In [ ]:
with open('result.sphere.12sext','rb') as f:
    result = pickle.load(f)
    
i=0
for item in lattice:
    if item.type=='multipole_thin':
        item['KL_sext']=20.0*result.x[i]
        i=i+1
        
impact.writeInputFile(beam,lattice)
impact.run(beam)
nLost1 = getLost()
indOff1,TBToff1 = impact.readTBT_integral(10)
indOn1,TBTon1 = impact.readTBT_integral_onMomentum(11)

In [ ]:
print(nLost0,nLost1)

In [ ]:
indOff0[:32]

In [ ]:
indOff1[:32]

In [ ]:
fig=plt.figure(figsize=(9,4))
plt.subplot(2,2,1)
for i in range(32):
    plt.plot(TBToff0[:,0,i],marker='.',ls='',alpha=0.7)
plt.subplot(2,2,2)
for i in range(32):
    plt.plot(TBToff0[:,1,i],marker='.',ls='',alpha=0.7)

In [ ]:
fig=plt.figure(figsize=(9,4))
plt.subplot(2,2,1)
for i in range(32):
    plt.plot(TBToff1[:,0,i],marker='.',ls='',alpha=0.7)
plt.subplot(2,2,2)
for i in range(32):
    plt.plot(TBToff1[:,1,i],marker='.',ls='',alpha=0.7)

In [ ]:
fig=plt.figure(figsize=(9,4))
plt.subplot(2,2,1)
for i in range(32):
    plt.plot((TBToff0[:,0,i]/TBToff0[0,0,i]-1.0),marker='.',ls='',alpha=0.7)
plt.subplot(2,2,2)
for i in range(32):
    plt.plot((TBToff0[:,1,i]/TBToff0[0,1,i]-1.0),marker='.',ls='',alpha=0.7)

In [ ]:
fig=plt.figure(figsize=(9,4))
plt.subplot(2,2,1)
for i in range(32):
    plt.plot((TBToff1[:,0,i]/TBToff1[0,0,i]-1.0),marker='.',ls='',alpha=0.7)
plt.subplot(2,2,2)
for i in range(32):
    plt.plot((TBToff1[:,1,i]/TBToff1[0,1,i]-1.0),marker='.',ls='',alpha=0.7)